In [ ]:
!pip install python-docx
from docx import Document
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
class Transcript:
    def __init__(self, transcript_id, base_pairs, length):
        self.transcript_id = transcript_id
        self.base_pairs = base_pairs
        self.length = length
from google.colab import drivep
drive.mount('/content/gdrive')
Foldername = '/content/gdrive/My Drive/'
doc = Document(Foldername + 'Base Pairs.docx')
transcripts = []
for table in doc.tables:
    for row in table.rows[1:]:
        cells = [cell.text.strip() for cell in row.cells]
        transcript = Transcript(*cells)
        transcripts.append(transcript)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
df = pd.DataFrame([(t.transcript_id, t.base_pairs, t.length) for t in transcripts],
                  columns=['Transcript_ID', 'Base_Pairs', 'Length'])
label_encoder = LabelEncoder()
df['Encoded_Length'] = label_encoder.fit_transform(df['Length'])
x_train, x_test, y_train, y_test = train_test_split(df[['Transcript_ID', 'Base_Pairs']], df['Encoded_Length'], test_size=0.25, random_state=3)
nb = BernoulliNB()
nb.fit(x_train['Base_Pairs'].str.extract(r'(\d+)'), y_train)
y_pred = nb.predict(x_test['Base_Pairs'].str.extract(r'(\d+)'))
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
results_df = pd.DataFrame({
    'True_Label': label_encoder.inverse_transform(y_test),
    'Predicted_Label': label_encoder.inverse_transform(y_pred),
    'Transcript_ID': x_test['Transcript_ID'],
    'Length': df.loc[y_test.index, 'Length']
})
print("Correct Predictions:")
print(results_df[results_df['True_Label'] == results_df['Predicted_Label']][['Transcript_ID', 'Length']])
print("\nIncorrect Predictions:")
print(results_df[results_df['True_Label'] != results_df['Predicted_Label']][['Transcript_ID', 'Length']])

Accuracy: 0.46153846153846156
Correct Predictions:
        Transcript_ID Length
12  ENST00000374231.8  Small
47  ENST00000576311.5  Small
31  ENST00000548101.1  Small
13  ENST00000374214.3  Small
48  ENST00000576834.2  Small
45  ENST00000571430.5  Small

Incorrect Predictions:
        Transcript_ID  Length
39  ENST00000629095.1  Medium
9   ENST00000580840.1  Medium
28  ENST00000713581.1   Large
6   ENST00000618887.2  Medium
36  ENST00000309922.7  Medium
23  ENST00000402774.8   Large
37  ENST00000313511.8  Medium
